In [1]:
from pathlib import Path
import datasets

In [2]:
import datetime
output_dir = Path('output') / 'cross_encoder' / datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [3]:
from xmen.reranking.cross_encoder import CrossEncoderTrainingArgs, CrossEncoderReranker

In [4]:
TASK_NAME = 'adapter_symptemist'

In [5]:
import adapters

In [6]:
rr = CrossEncoderReranker.load('phlobo/xmen-es-ce-medmentions', device=1)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
model = rr.model.model
model

XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250007, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768,

In [8]:
adapters.init(model)

In [9]:
adapter_args = adapters.training.AdapterArguments(train_adapter=True)

In [10]:
adapters.setup_adapter_training(model, adapter_args, TASK_NAME)

('adapter_symptemist', None)

In [12]:
from tqdm.auto import tqdm

params = 0
for p in tqdm(model.named_parameters()):    
    if p[1].requires_grad:
        print(p[0], p[1].numel())
        params += p[1].numel()

0it [00:00, ?it/s]

roberta.encoder.layer.0.output.adapters.adapter_symptemist.adapter_down.0.weight 36864
roberta.encoder.layer.0.output.adapters.adapter_symptemist.adapter_down.0.bias 48
roberta.encoder.layer.0.output.adapters.adapter_symptemist.adapter_up.weight 36864
roberta.encoder.layer.0.output.adapters.adapter_symptemist.adapter_up.bias 768
roberta.encoder.layer.1.output.adapters.adapter_symptemist.adapter_down.0.weight 36864
roberta.encoder.layer.1.output.adapters.adapter_symptemist.adapter_down.0.bias 48
roberta.encoder.layer.1.output.adapters.adapter_symptemist.adapter_up.weight 36864
roberta.encoder.layer.1.output.adapters.adapter_symptemist.adapter_up.bias 768
roberta.encoder.layer.2.output.adapters.adapter_symptemist.adapter_down.0.weight 36864
roberta.encoder.layer.2.output.adapters.adapter_symptemist.adapter_down.0.bias 48
roberta.encoder.layer.2.output.adapters.adapter_symptemist.adapter_up.weight 36864
roberta.encoder.layer.2.output.adapters.adapter_symptemist.adapter_up.bias 768
roberta

In [13]:
params

1485889

In [14]:
run = Path('/mnt/nfs/home/Florian.Borchert/runs/xmen/symptemist_benchmark/outputs/2024-02-07_16-15-59')
base_path = Path.home() / '.cache' / 'xmen' / 'symptemist'

In [15]:
from xmen import load_kb
kb = load_kb(base_path / 'symptemist.jsonl')

In [16]:
dataset = datasets.load_from_disk(run / '0-symptemist_benchmark_dataset')
candidates = datasets.load_from_disk(run / '0-symptemist_benchmark_candidates')

In [17]:
ce_dataset = CrossEncoderReranker.prepare_data(candidates, dataset, kb)

Context length: 128
Use NIL values: True


  0%|          | 0/2711 [00:00<?, ?it/s]

  0%|          | 0/2711 [00:00<?, ?it/s]

  0%|          | 0/2711 [00:00<?, ?it/s]

  0%|          | 0/773 [00:00<?, ?it/s]

  0%|          | 0/773 [00:00<?, ?it/s]

  0%|          | 0/773 [00:00<?, ?it/s]

  0%|          | 0/773 [00:00<?, ?it/s]

  0%|          | 0/773 [00:00<?, ?it/s]

  0%|          | 0/773 [00:00<?, ?it/s]

In [18]:
args = CrossEncoderTrainingArgs(num_train_epochs=20)

In [ ]:
import wandb

wandb.init(project="xmen")

try:
    rr.fit(args, ce_dataset['train'].dataset, ce_dataset['validation'].dataset, show_progress_bar=True, eval_callback=wandb.log, output_dir=output_dir, train_continue=True)
finally:
    if run := wandb.run:
        run.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: phlobo. Use `wandb login --relogin` to force relogin


model_name := bert-base-multilingual-cased
num_train_epochs := 20
fp16 := True
label_smoothing := False
rank_regularization := 1.0
train_layers := None
softmax_loss := True
random_seed := 42
learning_rate := 2e-05
Continue training


Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2711 [00:00<?, ?it/s]

2024-02-08 14:16:15 - EntityLinkingEvaluator: Evaluating the model on eval dataset after epoch 0:
2024-02-08 14:23:26 - Accuracy: 0.46571798188874514
2024-02-08 14:23:26 - Accuracy @ 5: 0.6869340232858991
2024-02-08 14:23:26 - Accuracy @ 64: 1.0
2024-02-08 14:23:26 - Baseline Accuracy: 0.4178525226390686
2024-02-08 14:23:26 - Save model to output/cross_encoder/20240208-134234


...

Iteration:   0%|          | 0/2711 [00:00<?, ?it/s]

2024-02-08 17:36:17 - EntityLinkingEvaluator: Evaluating the model on eval dataset after epoch 5:
2024-02-08 17:43:28 - Accuracy: 0.500646830530401
2024-02-08 17:43:28 - Accuracy @ 5: 0.6998706338939198
2024-02-08 17:43:28 - Accuracy @ 64: 1.0
2024-02-08 17:43:28 - Baseline Accuracy: 0.4178525226390686


Iteration:   0%|          | 0/2711 [00:00<?, ?it/s]

2024-02-08 18:16:14 - EntityLinkingEvaluator: Evaluating the model on eval dataset after epoch 6:
2024-02-08 18:23:25 - Accuracy: 0.5200517464424321
2024-02-08 18:23:25 - Accuracy @ 5: 0.7179818887451488
2024-02-08 18:23:25 - Accuracy @ 64: 1.0
2024-02-08 18:23:25 - Baseline Accuracy: 0.4178525226390686
2024-02-08 18:23:25 - Save model to output/cross_encoder/20240208-134234


Iteration:   0%|          | 0/2711 [00:00<?, ?it/s]